# Анализ базы данных запросами SQL

**Задача:** проанализировать базу данных крупного сервиса для чтения книг по подписке с целью создания нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'],
                                                         db_config['db']) 
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

- Выведем первые строки каждой таблицы: 

In [2]:
# таблица books
pd.io.sql.read_sql(
    """
    SELECT *
    FROM books
    LIMIT 10;
    """
    , con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


In [3]:
# таблица reviews
pd.io.sql.read_sql(
    """
    SELECT * 
    FROM reviews
    LIMIT 10;
    """
    , con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


In [4]:
# таблица ratings
pd.io.sql.read_sql(
    """
    SELECT * 
    FROM ratings
    LIMIT 10;
    """
    , con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


In [5]:
# таблица authors
pd.io.sql.read_sql(
    """
    SELECT * 
    FROM authors
    LIMIT 10;
    """
    , con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


- Посчитаем, сколько книг вышло после 1 января 2000 года;

In [6]:
pd.io.sql.read_sql(
    """
    SELECT COUNT(DISTINCT books) 
    FROM books
    WHERE publication_date > '2000-01-01';
    """
    , con = engine)

,count
0,819


819 книг вышло после 1 января 2000 года.

- Для каждой книги посчитаем количество обзоров и среднюю оценку;

In [7]:
pd.io.sql.read_sql(
    """
    SELECT 
        DISTINCT title, 
        AVG(rating) AS avg_rating, 
        COUNT(DISTINCT review_id) AS review_count 
    FROM books
    
    LEFT JOIN ratings ON ratings.book_id = books.book_id
    LEFT JOIN reviews ON reviews.book_id = books.book_id
    
    GROUP BY title, books.book_id -- есть книги с одним названием, но разным book_id --
    ORDER BY review_count DESC;
    """ 
    , con = engine)

,title,avg_rating,review_count
0,Twilight (Twilight #1),3.662500,7
1,Eat Pray Love,3.395833,6
2,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
4,Outlander (Outlander #1),4.125000,6
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,4.000000,0
996,Essential Tales and Poems,4.000000,0
997,Leonardo's Notebooks,4.000000,0
998,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,0


In [8]:
# максимальная оценка
pd.io.sql.read_sql(
    """
    SELECT 
    MAX(rating) 
    FROM ratings;
    """ 
    , con = engine)

,max
0,5


Больше всего обзоров у книги Memoirs of a Geisha и Twilight, при этом средние оценки у этих книг не самые высокие. В таблице есть книги без ревью. 

- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;

In [9]:
pd.io.sql.read_sql(
    """
    SELECT 
        COUNT(title), 
        publisher FROM books
    
    LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
    
    WHERE num_pages > 50
    
    GROUP BY publisher
    ORDER BY count DESC
    
    LIMIT 1;
    """
    , con = engine)

,count,publisher
0,42,Penguin Books


Больше всего книг выпустило издательство Penguin Books. 

- Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;

In [10]:
pd.io.sql.read_sql(
    """
    SELECT 
        author, 
        avg_rating
        
    FROM
        (SELECT 
            DISTINCT title, 
            author, 
            AVG(rating) AS avg_rating, 
            COUNT(DISTINCT rating_id) AS rating_cnt 
        
        FROM books
        
        LEFT JOIN ratings ON ratings.book_id = books.book_id
        LEFT JOIN authors ON authors.author_id = books.author_id
        
        GROUP BY title, author) AS count_r
    
    WHERE rating_cnt >= 50
    
    ORDER BY avg_rating DESC
    LIMIT 1;
    """
    , con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.414634


J.K. Rowling вместе с иллюстратором Mary GrandPré имеют наивысшую оценку среди самых популярных книг.

- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок;

In [11]:
pd.io.sql.read_sql(
    """
    SELECT AVG(count_review)
    FROM
    
    -- считаем пользователей, которые оставили больше 50 оценок --
    
        (SELECT username
         FROM
            (SELECT 
                DISTINCT username, 
                COUNT(DISTINCT rating_id) as count_ratings
            
             FROM ratings
            
             GROUP BY username) AS count_ratings
        
         WHERE count_ratings >=50) AS users_50
    
    LEFT JOIN 
    
    -- количество обзоров для каждого пользователя --
    
        (SELECT 
             DISTINCT username, 
             COUNT(DISTINCT review_id) as count_review
        
        FROM reviews
        
        GROUP BY username) AS count_review
    
    ON count_review.username = users_50.username
    """
    , con = engine)

,avg
0,24.222222


В среднем активные пользователи оставляют 24 обзора. Похоже что половину своих оценок они сопровождают комментариями.